## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: `id` 昇順（問題指定）
* `NOT IN` は本問題では不要（単純条件で完結）
* 判定は **「著者が自分の記事を閲覧したか」＝ `author_id = viewer_id`** 基準
  表示は列名 `id` で返す

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  著者が自分で書いた記事を **少なくとも 1 回** 閲覧した著者の ID を求め、
  列名 `id` で一意にして昇順で返す。

* 入力: `{{TABLES_OR_SCHEMAS}}`

  ```text
  Table: Views

  +-------------+---------+
  | Column Name | Type    |
  +-------------+---------+
  | article_id  | int     |
  | author_id   | int     |
  | viewer_id   | int     |
  | view_date   | date    |
  +-------------+---------+

  ・PK は定義されておらず、行は重複しうる。
  ・各行は「ある viewer が、ある author の記事を、ある日に閲覧した」ことを表す。
  ・`author_id = viewer_id` のとき「著者本人が自分の記事を閲覧した」ことを意味する。
  ```

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  ```text
  +------+
  | id   |
  +------+
  | ...  |
  +------+

  ・自分の記事を 1 回以上閲覧した著者の ID を重複なく返す。
  ・列名は必ず `id`。
  ・`id` 昇順でソート。
  ```

---

## 2) 最適解（単一クエリ）

> この問題は CTE やウィンドウ関数を使うまでもないため、
> **シンプルな `SELECT DISTINCT`** で解決するのが最適です。

```sql
SELECT DISTINCT
  author_id AS id
FROM Views v
WHERE v.author_id = v.viewer_id
ORDER BY id;

Runtime 337 ms
Beats 80.65%

```

### 補足：`GROUP BY` 版（等価解）

```sql
SELECT
  author_id AS id
FROM Views v
WHERE v.author_id = v.viewer_id
GROUP BY author_id
ORDER BY id;

Runtime 339 ms
Beats 75.65%

```

---

## 3) 要点解説

1. **「自分の記事を閲覧した行」の抽出**

   ```sql
   WHERE v.author_id = v.viewer_id
   ```

   * 著者と閲覧者が同一人物の行だけを残す。
   * 問題文の「自分の記事を閲覧した」をそのまま条件にした形。

2. **著者単位に一意化**

   ```sql
   SELECT DISTINCT author_id AS id
   ```

   * 同じ著者が同じ記事を複数回見ていても **1 行だけ** にする必要がある。
   * `DISTINCT`（または `GROUP BY author_id`）で **著者 ID をユニーク化**。

3. **出力列名と順序の揃え**

   ```sql
   author_id AS id
   ORDER BY id;
   ```

   * LeetCode の仕様に合わせ、列名は必ず `id` にリネーム。
   * ソート順も問題指定どおり `id` 昇順。

4. **`NOT IN` 回避について**

   * 今回は「除外」ではなく「該当する行の抽出」なので `NOT IN` 自体が不要。
   * そのため、`EXISTS` や `LEFT JOIN ... IS NULL` のようなパターンも出番なし。
   * 将来の類題では「A だが B ではない」のようなときに `NOT IN` の代わりに `NOT EXISTS` を使うとよい。

---

## 4) 計算量（概算）

* `Views` の行数を **N**、条件を満たす著者数を **A** とすると:

1. **WHERE 句のフィルタ (`author_id = viewer_id`)**

   * 全行スキャンで **O(N)**
   * インデックスがあれば `O(log N)` スキャン＋部分探索に近づくが、ここでは概ね線形とみなす。

2. **`DISTINCT` or `GROUP BY` による重複排除**

   * 著者 ID のユニーク化はソートベースの場合おおむね **O(A log A)**。
   * 著者数が全体よりかなり少ない場合、コストは小さく抑えられる。

3. **インデックスのヒント**

   * 実務なら以下のような複合インデックスを貼るとよいケースが多い:

     ```sql
     CREATE INDEX ON Views (author_id, viewer_id);
     ```

   * これにより `WHERE author_id = viewer_id` の探索が効率化される可能性がある。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Views テーブル 全閲覧ログ] --> B[著者 = 閲覧者
   の行だけ 
  WHERE author_id = viewer_id]
  B --> C[author_id ごとに一意化 
  DISTINCT / GROUP BY]
  C --> D[列名 id にリネーム 
  author_id AS id]
  D --> E[ORDER BY id ASC]
  E --> F["出力: 著者IDリスト
   (id 列のみ)"]
```

このクエリをそのまま PostgreSQL 16.6+ に流し込めば、問題の入出力仕様を満たす結果が得られます。

結論から言うと、そのクエリはこの問題に対して**ほぼ最適**で、SQL レベルでこれ以上大きく速くする余地はほとんどありません 👍

---

## 1. DISTINCT 版 vs GROUP BY 版について

```sql
SELECT DISTINCT
  author_id AS id
FROM Views v
WHERE v.author_id = v.viewer_id
ORDER BY id;
```

```sql
SELECT
  author_id AS id
FROM Views v
WHERE v.author_id = v.viewer_id
GROUP BY author_id
ORDER BY id;
```

PostgreSQL のプランナー的には、この2つはほぼ同じ実行計画になります。

* `DISTINCT` → 内部的に `GROUP BY` 相当の処理（`HashAggregate` or `Sort + Unique`）
* `GROUP BY author_id` → そのまま集約

LeetCode の実行環境の誤差（他ユーザーの並列実行状況など）を考えると、337ms vs 339ms は**完全に誤差レベル**です。
「Beats X%」の差も、ほぼ運ゲーみたいなものと考えてOKです。

---

## 2. ロジック面での改善余地

ロジックとしてはすでにミニマルで、これ以上削るところがありません。

* 条件は本質そのもの：`author_id = viewer_id`
* 余計な列もなく：`author_id` のみ
* 重複排除も最小限：`DISTINCT` / `GROUP BY`

この問題に対して **JOIN もサブクエリも不要** なので、今の形が「教科書的な模範解答」です。

---

## 3. 実務 PostgreSQL でのチューニング余地（LeetCode 以外の話）

LeetCode 環境では DDL（INDEX 作成など）はできない前提なので、**もし実務で巨大テーブルに対して同じクエリを投げるなら**という観点での改善案を挙げておきます。

### (1) インデックス追加

```sql
CREATE INDEX CONCURRENTLY idx_views_author_viewer
  ON Views (author_id, viewer_id);
```

* `WHERE author_id = viewer_id` 条件での探索がかなり速くなる可能性があります。
* 条件に完全一致する行が少ないなら、インデックススキャン＋ビットマップヒープスキャンなどが効きやすくなります。

もっと尖らせるなら、部分インデックスも検討できます：

```sql
CREATE INDEX CONCURRENTLY idx_views_self_view
  ON Views (author_id)
  WHERE author_id = viewer_id;
```

* 「自分の記事を見た行だけ」をインデックスに持つので、対象行が少ないほど効きやすいです。

### (2) 統計情報と ANALYZE

巨大テーブルなら、`ANALYZE Views;` で統計情報を最新化しておくことで

* プランナーが「`author_id = viewer_id` を満たす行の割合」を適切に見積もれる
* 結果として Seq Scan / Index Scan / Bitmap Scan の選択が賢くなる

…といったプラン最適化が期待できます。

---

## 4. 書き方の微修正（ほぼ好み）

今の書き方も十分読みやすいですが、個人的には LeetCode のような単一テーブル問題なら、もう少しだけミニマルにしても良いかな、くらいです。

```sql
SELECT DISTINCT
  author_id AS id
FROM Views
WHERE author_id = viewer_id
ORDER BY id;

Runtime 330 ms
Beats 95.81%

```

* テーブルエイリアス（`v`）は使わなくても混乱しないので省略可
* 実行計画・パフォーマンスは変わりません（完全にスタイルの話）

---

## 5. まとめ

* あなたのクエリは **ロジック的にもパフォーマンス的にもベストクラス** です ✅
* `DISTINCT` 版と `GROUP BY` 版の差はほぼなく、337ms vs 339ms の違いは誤差と考えてOK。
* もし実務でテーブルが数千万行以上あるようなケースなら、

  * `(author_id, viewer_id)` の複合インデックス
  * or `WHERE author_id = viewer_id` の部分インデックス
  * ＋ `ANALYZE`
    といったチューニングが効いてきます。

この問題に関しては、**SQL の書き方をいじってもほぼこれ以上は速くならない**領域に入っているので、「もう十分やり切った」と思って大丈夫です 🙆‍♂️
